Student Answer -> Bert encoding\
Question -> BERT encoding ------.->cross attention

In [1]:
from bert_embedding import BertEmbedding


In [15]:
from torch import nn
import torch
import torch.nn.functional as F

In [3]:
import numpy as np
import math

In [4]:
bert_abstract = """We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers.
 Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations by jointly conditioning on both left and right context in all layers.
 As a result, the pre-trained BERT representations can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications. 
BERT is conceptually simple and empirically powerful. 
It obtains new state-of-the-art results on eleven natural language processing tasks, including pushing the GLUE benchmark to 80.4% (7.6% absolute improvement), MultiNLI accuracy to 86.7 (5.6% absolute improvement) and the SQuAD v1.1 question answering Test F1 to 93.2 (1.5% absolute improvement), outperforming human performance by 2.0%."""

In [5]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [6]:
student_ans = "Sky is red"
question = "What is the colour of the sky?"
reference_ans ="Sky appears blue"

In [7]:
def getBertEncoding(paragraph):
        sentences = paragraph.split('.')
        bert_embedding = BertEmbedding()
        result = bert_embedding(sentences)
        return result

In [8]:
K = getBertEncoding(student_ans)
Q = getBertEncoding(question)

In [9]:
K_values = K[0][1]
Q_values = Q[0][1]

In [10]:
K_values = np.array(K_values)
Q_values = np.array(Q_values)

In [17]:
K = torch.from_numpy(K_values)
Q = torch.from_numpy(Q_values)

In [13]:
cross_attention = attention(Q,K,K)

In [133]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

tensor([[0.7137, 0.1191, 0.1672],
        [0.0628, 0.8501, 0.0871],
        [0.1534, 0.6523, 0.1942],
        [0.2714, 0.3641, 0.3644],
        [0.1581, 0.6745, 0.1674],
        [0.1706, 0.6694, 0.1599],
        [0.5924, 0.2356, 0.1720],
        [0.1272, 0.2982, 0.5746]])

In [13]:
K_values.shape

(3, 768)

In [3]:
class FeedForwardLayer(nn.Module):
    def __init__(self,inp_dim,hid_dim,dropout = 0.1):
        super().__init__()
        self.inp_dim = inp_dim
        self.hid_sim = hid_dim
        self.hidden = nn.Linear(inp_dim,hid_dim)
        self.output = nn.Linear(hid_dim,inp_dim)
        self.relu = nn.ReLU
        self.dropout = nn.Dropout
        
    def forward(self,x):
        x = self.hidden(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.output(x)
        return x

In [4]:
class EncoderBlock(nn.Module):
    def __init__(self,Value,Query,attentionBlock,feedForwardBlock,normBlock):
        super().__init__()
        self.value = Value
        self.query = Query
        self.attentionBlock = attentionBlock
        self.feedForwardBlock = feedForwardBlock
        self.normBlock = normBlock

    def forward(self,x):

        att = self.attentionBlock(x)
        x = self.normBlock(att,x)
        ff = lambda(x, self.feedForwardBlock(x))
        x = normBlock(ff)

        return x

    

In [ ]:
class EncoderModule(nn.Module):
    def __init__(self,Question,Answer)

In [ ]:
class RepresentationModule(nn.Module):
    def __init__

In [19]:
K.shape

torch.Size([3, 768])

In [23]:
multihead_attn = nn.MultiheadAttention(embed_dim = 768, num_heads=3)

In [36]:
Value = K.reshape(3,1,768)


In [58]:
Query = Q.reshape(8,1,768)

In [59]:
Query

tensor([[[ 0.5666,  0.0905, -0.3259,  ...,  0.1991,  0.3866,  0.0199]],

        [[ 0.2259,  0.1880,  0.2643,  ..., -0.1996,  0.0378,  0.6731]],

        [[ 0.4941, -0.0662,  0.1847,  ...,  0.0330, -0.1765,  0.4608]],

        ...,

        [[ 0.4937,  0.1258,  0.0544,  ..., -0.1227,  0.0570, -0.1065]],

        [[ 0.8392,  0.2743, -0.0246,  ...,  0.1315,  0.3748, -0.1369]],

        [[-0.0101,  0.0074, -0.9819,  ...,  0.0519,  0.2797, -0.2009]]])

In [56]:
out,att =multihead_attn(key =Value,value = Value,query = Query)

In [54]:
out.shape

torch.Size([3, 1, 768])

In [57]:
out

tensor([[[ 0.0201,  0.0883,  0.0876,  ...,  0.2936,  0.1124, -0.2299]],

        [[ 0.0235,  0.0832,  0.0853,  ...,  0.3003,  0.1076, -0.2377]],

        [[ 0.0253,  0.0837,  0.0876,  ...,  0.3009,  0.1112, -0.2433]],

        ...,

        [[ 0.0266,  0.0802,  0.0843,  ...,  0.3021,  0.1068, -0.2465]],

        [[ 0.0245,  0.0842,  0.0832,  ...,  0.2978,  0.1063, -0.2371]],

        [[ 0.0246,  0.0870,  0.0877,  ...,  0.3002,  0.1140, -0.2309]]],
       grad_fn=<AddBackward0>)